<a href="https://colab.research.google.com/github/YashsTiwari/ML_DL_revision/blob/main/Coding_AlexNet_using_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d ifigotin/imagenetmini-1000

Dataset URL: https://www.kaggle.com/datasets/ifigotin/imagenetmini-1000
License(s): unknown
100% 3.91G/3.92G [01:02<00:00, 60.5MB/s]
100% 3.92G/3.92G [01:02<00:00, 66.9MB/s]


In [3]:
import zipfile
zip_ref=zipfile.ZipFile('/content/imagenetmini-1000.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [5]:
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [6]:
train_dataset=datasets.ImageFolder(root='/content/imagenet-mini/train',transform=transform)
val_dataset=datasets.ImageFolder(root='/content/imagenet-mini/val',transform=transform)

In [7]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [12]:
class AlexNet(nn.Module):
  def __init__(self,input_features):
    super().__init__()
    self.features=nn.Sequential(
       nn.Conv2d(input_features, 96, kernel_size=11, stride=4, padding=0),
       nn.ReLU(),
       nn.MaxPool2d(kernel_size=3,stride=2),
       nn.Conv2d(96, 256, kernel_size=5, padding=0),
       nn.ReLU(),
       nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
       nn.Conv2d(256, 384, kernel_size=3, padding=0),
       nn.ReLU(),
       nn.Conv2d(384, 384, kernel_size=3, padding=1),
       nn.ReLU(),
       nn.Conv2d(384, 256, kernel_size=3, padding=1),
       nn.ReLU(),
       nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
    )
    self.classifier=nn.Sequential(
       nn.Flatten(),
       nn.Linear(4*4*256, 4096),
       nn.ReLU(),
       nn.Dropout(0.5),
       nn.Linear(4096, 4096),
       nn.ReLU(),
       nn.Dropout(0.5),
       nn.Linear(4096, 1000)
    )
  def forward(self, x):
     x=self.features(x)
     x=self.classifier(x)
     return x

In [13]:
import torch.optim as optim
model=AlexNet(3)
lr = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=5e-4)

In [14]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

In [11]:
dummy = torch.randn(1, 3, 227, 227).to(device)
print(model.features(dummy).shape)

torch.Size([1, 256, 4, 4])


In [15]:
def train(model, train_loader, val_loader, optimizer, criterion, device, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = outputs.max(1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        accuracy = 100 * correct / total

        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {accuracy:.2f}%")

In [19]:
train(model, train_loader, val_loader, optimizer, criterion, device, epochs=10)

Epoch [1/10] | Train Loss: 6.8111 | Val Loss: 6.8421 | Val Acc: 0.08%
Epoch [2/10] | Train Loss: 6.7959 | Val Loss: 6.8371 | Val Acc: 0.10%
Epoch [3/10] | Train Loss: 6.7819 | Val Loss: 6.8322 | Val Acc: 0.10%
Epoch [4/10] | Train Loss: 6.7716 | Val Loss: 6.8152 | Val Acc: 0.20%
Epoch [5/10] | Train Loss: 6.7556 | Val Loss: 6.8071 | Val Acc: 0.25%
Epoch [6/10] | Train Loss: 6.7351 | Val Loss: 6.7855 | Val Acc: 0.31%
Epoch [7/10] | Train Loss: 6.7002 | Val Loss: 6.7528 | Val Acc: 0.25%
Epoch [8/10] | Train Loss: 6.6422 | Val Loss: 6.6814 | Val Acc: 0.43%
Epoch [9/10] | Train Loss: 6.5808 | Val Loss: 6.6562 | Val Acc: 0.66%
Epoch [10/10] | Train Loss: 6.5296 | Val Loss: 6.5963 | Val Acc: 0.66%


## This is just a basic version, more optimisations can be done